# Fit the far-field diffraction geometry using a powder calibrant with dx dy spatial files

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
import copy
import silx.io
import scipy.ndimage
import numpy as np
import fabio
import matplotlib.pyplot as plt
import h5py
import matplotlib
import ImageD11.columnfile
import ImageD11.unitcell
import ImageD11.transformer
import ImageD11.frelon_peaksearch
from ImageD11 import parameters
from ImageD11.cImageD11 import s_1, s_I, s_sI, s_fI
from ImageD11.nbGui import segmenter_gui
from ImageD11.nbGui import fit_geometry
from ImageD11.parameters import AnalysisSchema, parameters
%matplotlib ipympl

In [ ]:
# Set up the file paths. Edit this if you are not at ESRF or not using the latest data policy.
dataroot, analysisroot = segmenter_gui.guess_ESRF_paths() 

if len(dataroot)==0:
    print("Please fix in the dataroot and analysisroot folder names above!!")
    
print('dataroot =',repr(dataroot))
print('analysisroot =',repr(analysisroot))

In [ ]:
# List the samples available:
segmenter_gui.printsamples(dataroot)

In [ ]:
# USER: Decide which sample
sample = 'CeO2'

In [ ]:
# List the datasets for that sample:
segmenter_gui.printdatasets( dataroot, sample )

In [ ]:
# USER: Decide which dataset
dataset = "rot"

In [ ]:
# USER: which detector are you using?
detector = 'frelon3'

In [ ]:
# USER: Which scan is the calibration?
scan = '1.1'

In [ ]:
ds = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                        analysisroot=analysisroot,
                                        sample=sample,
                                        dset=dataset,
                                        detector=detector)

frames = silx.io.get_data( f"silx:{ds.masterfile}::{scan}/measurement/{ds.detector}")
print(frames.shape)

In [ ]:
# now specify where the dark scan is
# if we didn't collect one, change the below to False

have_dark_scan = False

if have_dark_scan:
    
    dark_sample = 'CeO2'
    dark_dataset = "dark"
    dark_scan = '1.1'
    
    ds_dark = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                                analysisroot=analysisroot,
                                                sample=dark_sample,
                                                dset=dark_dataset,
                                                detector=detector)

    dark = silx.io.get_data(f"silx:{ds_dark.masterfile}::{dark_scan}/measurement/{ds_dark.detector}" )[:].mean(axis=0, dtype=np.float32)
    print(dark.shape)

In [ ]:
# Try to determine which spatial calibration files to use depending on the detector

if 'frelon' in detector:
    serial = silx.io.get_data( f"silx:{ds.masterfile}::{scan}/instrument/{detector}/camera_settings/camera_serial" )
    o11 = '1'
    print("Detector serial", serial )
    if serial == "21":
        flat = silx.io.get_data( f"fabio:/data/id11/3dxrd/inhouse/Frelon21/F21_flat_july18.edf" )
        e2dxfile = "/data/id11/3dxrd/inhouse/Frelon21/F21dxnew.edf"
        e2dyfile = "/data/id11/3dxrd/inhouse/Frelon21/F21dynew.edf"
        maskfile = "/data/id11/3dxrd/inhouse/Frelon21/F21_flat_Oct2016.msk"
        pixelsize = float(fabio.open(e2dxfile).header['PSize_1'].split()[0])*1e6 # microns

    elif serial == "36": 
        flat = silx.io.get_data( f"fabio:/data/id11/3dxrd/inhouse/Frelon36/F36_Nov2023.edf" )
        flat = flat / flat[500:-500, 500:-500].mean()
        e2dxfile = "/data/id11/3dxrd/inhouse/Frelon36/f36dx.edf"
        e2dyfile = "/data/id11/3dxrd/inhouse/Frelon36/f36dy.edf"
        maskfile = "/data/id11/3dxrd/inhouse/Frelon36/frelon36_mask_20240117.edf"
        pixelsize = float(fabio.open(e2dxfile).header['PSize_1'].split()[0])*1e6 # microns

    elif serial == "29":
        flat = silx.io.get_data( f"fabio:/data/id11/3dxrd/inhouse/Frelon4M/c29_Agfilt_flat-dark.edf" )
        e2dxfile = "/data/id11/3dxrd/inhouse/Frelon4M/F4M_EO_dx.edf"
        e2dyfile = "/data/id11/3dxrd/inhouse/Frelon4M/F4M_EO_dy.edf"
        maskfile = "/data/id11/3dxrd/inhouse/Frelon4M/c29_Agfilt_flat-dark-mask.edf"
        pixelsize = float(fabio.open(e2dxfile).header['PSize_1'].split()[0])*1e6 # microns
    else:
        print('Not sure what Frelon your detector is! You should manually provide the flat field, e2dxfile, e2dyfile, pixelsize, maskfile')
    print('Pixel size', pixelsize )
    print( "Average flat, should be close to 1 : ",flat[100:-100, 100:-100].mean() )
    
    if have_dark_scan:
        cor = ( frames - dark[ None, :, :] ).mean( axis = 0, dtype = np.float32 ) / flat
    else:
        cor = frames.mean( axis = 0, dtype = np.float32 ) / flat
    
elif 'eiger' in detector: # eiger
    cor = frames.mean( axis = 0, dtype = np.float32 )
    o11 = '-1'
    e2dxfile = "/data/id11/nanoscope/Eiger/e2dx_E-08-0144_20240205.edf"
    e2dyfile = "/data/id11/nanoscope/Eiger/e2dy_E-08-0144_20240205.edf"
    maskfile = "/data/id11/nanoscope/Eiger/eiger_mask_E-08-0144_20240205.edf"
    pixelsize = float(silx.io.get_data(f"silx:{ds.masterfile}::{scan}/instrument/{ds.detector}/x_pixel_size" )) * 1e6  # microns
else:
    print('Unknown detector! Must manually specify')

In [ ]:
def guess_median_bg( img, size=91 ):
    m0 = scipy.ndimage.median_filter( cor, (size,1))
    m1 = scipy.ndimage.median_filter( cor, (1,size))
    bg = np.where(m0<m1,m0,m1)
    return bg

In [ ]:
# load mask image

mask_frame = fabio.open(maskfile).data

fig, ax = plt.subplots(figsize=(10, 10), constrained_layout=True)
ax.imshow(mask_frame, interpolation='nearest')
ax.set_title('Detector mask')
plt.show()

In [ ]:
# USER: choose a threshold that separates peaks from the air scattering background
if 'eiger' in detector:
    default_lowcut = 11
elif 'frelon' in detector:
    if serial == '36':
        default_lowcut = 150
    else:
        default_lowcut = 500
else:
    default_lowcut = 100
lowcut = default_lowcut  # low cutoff for masking - this will probably need changing, check next plot to verify
mask = ~mask_frame & ( ( cor < 4e9 ) & (cor > lowcut ) )

bg = guess_median_bg( cor )

signal = ((cor - bg )*mask).clip(0,None)

In [ ]:
f,(a,b) = plt.subplots( 2, 2, figsize=(8, 6), constrained_layout=True )
f.colorbar( a[1].imshow( cor * mask, norm='log', vmin=0.5, vmax=500, interpolation='nearest' ) )
f.colorbar( b[1].imshow( signal, norm='log', vmin=0.5, vmax=500, interpolation='nearest' ) )
a[1].set_title('Correction')
a[0].plot( (cor * mask)[ 1024, : ] )
a[0].plot( (cor * mask)[ :, 1024 ] )
a[0].set(xlabel='pixel', ylabel='intensity')
b[0].plot( signal[ 1024, : ] )
b[0].plot( signal[ :, 1024 ] )
b[1].set_title('Signal to peaksearch')
b[0].set(xlabel='pixel', ylabel='intensity', ylim=(0, None) )
plt.show()

In [ ]:
# Old way, using command line:
# !powderimagetopeaks.py CeO2_for_S5_blue_rot_clean.edf CeO2_for_S5_blue_rot_clean0000.edf CeO2_for_S5_blue_rot_clean0001.edf 1024 1024
def powderimagetopeaks( img, ci = 1024, cj = 1024 ):
    # cuts into 1 degree bins
    si, sj = img.shape
    i, j = np.mgrid[0:si,0:sj]
    phi = np.arctan2( i - ci, j - cj ) * 361 / np.pi
    return (phi%2).astype(int) == 0

In [ ]:
peak_mask = powderimagetopeaks(signal)

worker = ImageD11.frelon_peaksearch.worker(None, None)
worker.threshold = 0

pks = np.concatenate( (
     worker.peaksearch( signal * peak_mask ),     # even degrees
     worker.peaksearch( signal * (1-peak_mask)), # odd degrees
    ), axis = 0 )
print(f'Found {pks.shape[0]} peaks')

In [ ]:
colfile = ImageD11.columnfile.colfile_from_dict( {
    "s_raw" : pks[ :, s_sI ] / pks[ :, s_I ],
    "f_raw" : pks[ :, s_fI ] / pks[ :, s_I ],
    "Number_of_pixels" : pks[ :, s_1 ], 
    "sum_intensity" : pks[ :, s_I ],
    "omega" : np.zeros( len(pks), float ),
} )

In [ ]:
f, a = plt.subplots(1,2, figsize=(8,5), constrained_layout=True)
a[0].plot( colfile.f_raw, colfile.s_raw, ".", ms = 1)
a[0].set( ylabel="detector slow direction", xlabel="detector fast direction", aspect='equal')
a[1].plot( colfile.Number_of_pixels, colfile.sum_intensity, ".", ms =1 )
a[1].set( xlabel="Number of pixels", ylabel="sum intensity", yscale='log', xscale='log')
plt.show()

In [ ]:
colfile = ImageD11.blobcorrector.eiger_spatial( e2dxfile, e2dyfile )(colfile)

In [ ]:
def auto_guess_distance(masterfile, scan):
    """
    Automatically guess the distance from the masterfile
    """
    possible_distance_motors = ['ffdtx1', 'frelx']
    distance_um = None
    for mot in possible_distance_motors:
        try:
            distance_um = float(silx.io.get_data(f"silx:{masterfile}::{scan}/instrument/positioners/{mot}" )) * 1e3  # microns
        except ValueError:
            continue
    if distance_um is None:
        raise ValueError("Couldn't find distance!")
    
    return distance_um

In [ ]:
# guess the detector distance in um
# you can also manually specify
distance_guess = auto_guess_distance(ds.masterfile, scan)  # detector distance in um

# 'Ag': 25.514, 'Sn': 29.2001, 'Nd': 43.5689, 'Gd': 50.2391, 'Hf': 65.3508, 'W' : 69.525, 'Pt': 78.3948, 'Pb': 88.0045
# here enter the lattice parameters and spacegroup of the calibrant
# at ID11 we use CeO2 674b

a_calibrant = 5.4115260
spacegroup_calibrant = 225
econst = 12.398423  # energy conversion - don't change
energy = 43.5689  # the energy in keV

In [ ]:
# get initial parameters

trans = ImageD11.transformer.transformer()
initial_parameters = trans.parameterobj
initial_parameters.set('cell__a', a_calibrant)
initial_parameters.set('cell__b', a_calibrant)
initial_parameters.set('cell__c', a_calibrant)
initial_parameters.set('cell_lattice_[P,A,B,C,I,F,R]', spacegroup_calibrant)
initial_parameters.set('distance', distance_guess)
initial_parameters.set('o11', int(o11))
initial_parameters.set('wavelength', econst/energy)
initial_parameters.set('y_size', pixelsize)
initial_parameters.set('z_size', pixelsize)

initial_parameters.get_parameters()

In [ ]:
ui = ImageD11.nbGui.fit_geometry.FitGeom( )
ui.setfiltered(colfile)
ui.parameterobj = copy.deepcopy(initial_parameters)
ui.fitGui()

In [ ]:
# now save the parameter file to disk

final_par_file = 'powder_pars.par'
ui.saveparameters(final_par_file)

__We can now make a new-style parameter file, with Si added as a phase, for the single-crystal calibration notebooks.__

In [ ]:
asc = AnalysisSchema.from_old_pars_file(final_par_file)
si_ucell = ImageD11.unitcell.unitcell([5.43094, 5.43094, 5.43094, 90.0, 90.0, 90.0], 227)
asc.add_phase_from_unitcell(phase_name='Si', unitcell=si_ucell)
asc.save('pars.json')